In [25]:
from pathlib import Path
import xarray as xr
from utils import model_info, general
import scipy

In [20]:
dpath_str = "/N/slate/jmelms/projects/dcmip2025_idealized_tests/experiments/H.energy_balance/data/E3SM_runs/v2.1.WCYCLSSP370_20180201-20180401_12n32p_original_20251013/run/v2.1.WCYCLSSP370_20180201-20180401_12n32p_original_20251013.eam.h1.2018-02-01-00000.nc"
dpath = Path(dpath_str)
print("Retrieving data")

Retrieving data


In [21]:
full_ds = xr.open_dataset(dpath)
full_ds

<xarray.Dataset> Size: 3GB
Dimensions:              (ncol: 21600, lev: 72, ilev: 73, cosp_prs: 7, nbnd: 2,
                          cosp_tau: 7, cosp_ht: 40, cosp_sr: 15,
                          cosp_htmisr: 16, cosp_tau_modis: 7, cosp_reffice: 6,
                          cosp_reffliq: 6, time: 120, cosp_scol: 10, cosp_sza: 5)
Coordinates: (12/13)
  * lev                  (lev) float64 576B 0.1238 0.1828 0.2699 ... 993.8 998.5
  * ilev                 (ilev) float64 584B 0.1 0.1477 0.218 ... 997.0 1e+03
  * cosp_prs             (cosp_prs) float64 56B 9e+04 7.4e+04 ... 2.45e+04 9e+03
  * cosp_tau             (cosp_tau) float64 56B 0.15 0.8 2.45 ... 41.5 100.0
  * cosp_scol            (cosp_scol) int32 40B 1 2 3 4 5 6 7 8 9 10
  * cosp_ht              (cosp_ht) float64 320B 1.896e+04 1.848e+04 ... 240.0
    ...                   ...
  * cosp_sza             (cosp_sza) float64 40B 0.0 20.0 40.0 60.0 80.0
  * cosp_htmisr          (cosp_htmisr) float64 128B 0.0 250.0 ... 1.8e+04
  * cosp_tau_modis       (cosp_tau_modis) float64 56B 0.15 0.8 ... 41.5 100.0
  * cosp_reffice         (cosp_reffice) float64 48B 5e-06 1.5e-05 ... 7.5e-05
  * cosp_reffliq         (cosp_reffliq) float64 48B 4e-06 9e-06 ... 2.5e-05
  * time                 (time) object 960B 2018-02-01 00:00:00 ... 2018-03-0...
Dimensions without coordinates: ncol, nbnd
Data variables: (12/111)
    lat                  (ncol) float64 173kB ...
    lon                  (ncol) float64 173kB ...
    area                 (ncol) float64 173kB ...
    hyam                 (lev) float64 576B ...
    hybm                 (lev) float64 576B ...
    P0                   float64 8B ...
    ...                   ...
    Z500                 (time, ncol) float32 10MB ...
    Z600                 (time, ncol) float32 10MB ...
    Z700                 (time, ncol) float32 10MB ...
    Z850                 (time, ncol) float32 10MB ...
    Z925                 (time, ncol) float32 10MB ...
    ZBOT                 (time, ncol) float32 10MB ...
Attributes: (12/19)
    ne:                30
    fv_nphys:          2
    title:             EAM History file information
    source:            E3SM Atmosphere Model
    source_id:         979fb1cfb8
    product:           model-output
    ...                ...
    institution_id:    E3SM-Project
    institution:       LLNL (Lawrence Livermore National Laboratory, Livermor...
    contact:           e3sm-data-support@llnl.gov
    initial_file:      v21.LR.SMYLE_IC.2018-02.01.eam.i.2018-02-01-00000.nc
    topography_file:   /N/project/obrienta_startup/datasets/CESMinputfiles/at...
    time_period_freq:  hour_6

In [26]:
vars_in_ds = (set(map(str.upper, model_info.MASTER_VARIABLES_NAMES)) & set(full_ds.data_vars)) | {"TMQ", "PS", "PHIS", "U050", "V050", "T050", "Z050", "Q050", "lat"}
e3sm_to_cds = {var: var.lower() for var in vars_in_ds}
e3sm_to_cds["U050"] = "u50" # 50 hPa zonal wind
e3sm_to_cds["V050"] = "v50" # 50 hPa meridional wind
e3sm_to_cds["T050"] = "t50" # 50 hPa temperature
e3sm_to_cds["Z050"] = "z50" # 50 hPa geopotential height
e3sm_to_cds["Q050"] = "q50" # 50 hPa specific humidity
e3sm_to_cds["PS"] = "sp" # surface pressure
e3sm_to_cds["PHIS"] = "z" # surface geopotential
e3sm_to_cds["TMQ"] = "tcwv" # total column water vapor
ds = full_ds[vars_in_ds].rename(e3sm_to_cds)
ds

<xarray.Dataset> Size: 653MB
Dimensions:  (time: 120, ncol: 21600)
Coordinates:
  * time     (time) object 960B 2018-02-01 00:00:00 ... 2018-03-02 18:00:00
Dimensions without coordinates: ncol
Data variables: (12/64)
    v925     (time, ncol) float32 10MB ...
    q850     (time, ncol) float32 10MB ...
    u500     (time, ncol) float32 10MB ...
    z925     (time, ncol) float32 10MB ...
    v100     (time, ncol) float32 10MB ...
    z200     (time, ncol) float32 10MB ...
    ...       ...
    t925     (time, ncol) float32 10MB ...
    u100     (time, ncol) float32 10MB ...
    u300     (time, ncol) float32 10MB ...
    t150     (time, ncol) float32 10MB ...
    z850     (time, ncol) float32 10MB ...
    t600     (time, ncol) float32 10MB ...
Attributes: (12/19)
    ne:                30
    fv_nphys:          2
    title:             EAM History file information
    source:            E3SM Atmosphere Model
    source_id:         979fb1cfb8
    product:           model-output
    ...                ...
    institution_id:    E3SM-Project
    institution:       LLNL (Lawrence Livermore National Laboratory, Livermor...
    contact:           e3sm-data-support@llnl.gov
    initial_file:      v21.LR.SMYLE_IC.2018-02.01.eam.i.2018-02-01-00000.nc
    topography_file:   /N/project/obrienta_startup/datasets/CESMinputfiles/at...
    time_period_freq:  hour_6

In [23]:
for var in full_ds.data_vars:
    if var not in vars_in_ds:
        print(f"{var}: {full_ds[var].attrs.get('long_name', '')}")

lat: latitude
lon: longitude
area: physics grid areas
hyam: hybrid A coefficient at layer midpoints
hybm: hybrid B coefficient at layer midpoints
P0: reference pressure
hyai: hybrid A coefficient at layer interfaces
hybi: hybrid B coefficient at layer interfaces
cosp_prs_bnds: 
cosp_tau_bnds: 
cosp_ht_bnds: 
cosp_sr_bnds: 
cosp_htmisr_bnds: 
cosp_tau_modis_bnds: 
cosp_reffice_bnds: 
cosp_reffliq_bnds: 
date: current date (YYYYMMDD)
datesec: current seconds of current date
time_bnds: time interval endpoints
date_written: 
time_written: 
ndbase: base day
nsbase: seconds of base day
nbdate: base date (YYYYMMDD)
nbsec: seconds of base date
mdt: timestep
ndcur: current day (from base day)
nscur: current seconds of current day
co2vmr: co2 volume mixing ratio
ch4vmr: ch4 volume mixing ratio
n2ovmr: n2o volume mixing ratio
f11vmr: f11 volume mixing ratio
f12vmr: f12 volume mixing ratio
sol_tsi: total solar irradiance
nsteph: current timestep
LANDFRAC: Fraction of sfc area covered by land
Q: Sp

In [ ]:
# Set constants
cp = 1005.0  # J/kg/K
g = 9.81  # m/s^2
Lv = 2.26e6  # J/kg
sb_const = 5.670374419e-8  # W/m^2/K^4, from https://physics.nist.gov/cgi-bin/cuu/Value?sigma

In [ ]:
## Step 4a: Compute a mask of the p-levels above the surface for integration ###
sp = ds["sp"] / 100
sp_expanded = sp.expand_dims(dim={"level": ds.sizes["level"]}, axis=-1)
expand_dict = {dim: ds.sizes[dim] for dim in ds.dims if dim != "level"}
levs_expanded = ds["level"].expand_dims(dim=expand_dict)
mask = levs_expanded <= sp_expanded
ds["terrain_mask"] = mask

### Step 4b: Get pressure for integration ###
pa = 100 * ds.level.values # convert to Pa from hPa, used for integration

### Step 4c: Calculate total energy components ###
# sensible heat
sensible_heat = cp * ds["T"]
# latent heat - this is already column-integrated
latent_heat = Lv * ds["TCW"]
# geopotential energy
geopotential_energy = ds["Z"] # geopotential energy is already in J/kg, no need to multiply by g
# kinetic energy
kinetic_energy = 0.5 * ds["U"] ** 2 + 0.5 * ds["V"] ** 2

### Step 4d: Calculate total energy by adding all components ###
# total energy minus latent heat
dry_total_energy = sensible_heat + geopotential_energy + kinetic_energy
dry_total_energy = dry_total_energy.where(mask, 0.0) # set values below surface to 0
# column integration
print(f"Integrating dry total energy with shape {dry_total_energy.shape} and pa with shape {pa.shape}")
dry_total_energy_column = (1 / g) * scipy.integrate.trapezoid( # add nan-safe version
    dry_total_energy, pa, axis=3
)

# sum
ds["VAR_TE"] = (expand_dict, dry_total_energy_column + latent_heat.values)
ds["VAR_TE"] = ds["VAR_TE"].assign_attrs(
    {"units": "J/m^2", "long_name": "Total Energy"}
)

### Step 4e: Weight by latitude ####
# get latitude weighted total energy (time, ensemble)
ds["LW_TE"] = general.latitude_weighted_mean(ds["VAR_TE"], ds.latitude)
ds["LW_TE"].assign_attrs(
    {"units": "J/m^2", "long_name": "Latitude-Weighted Total Energy"}
)